#Data Lake Storage Gen2 の初期化

In [0]:
service_credential = dbutils.secrets.get(scope="key-vault-secret",key="access-key")

abfs_container_name = "<container_name>"
abfs_account = "<abfs_account>"
tenant_id = "<tenant_id>"
client_id = "<client_id>"

spark.conf.set(f"fs.azure.account.auth.type.{abfs_account}", "OAuth")
spark.conf.set(f"fs.azure.account.oauth.provider.type.{abfs_account}", "org.apache.hadoop.fs.azurebfs.oauth2.ClientCredsTokenProvider")
spark.conf.set(f"fs.azure.account.oauth2.client.id.{abfs_account}.dfs.core.windows.net", client_id)
spark.conf.set(f"fs.azure.account.oauth2.client.secret.{abfs_account}.dfs.core.windows.net", service_credential)
spark.conf.set(f"fs.azure.account.oauth2.client.endpoint.{abfs_account}.dfs.core.windows.net", f"https://login.microsoftonline.com/{tenant_id}/oauth2/token")
spark.conf.set("spark.databricks.unityCatalogOnlyModel", "true")

dl2Uri = f"abfss://{abfs_container_name}@{abfs_account}"

##入力操作

###ファイルリスト

In [0]:
#仕入先
dbutils.fs.ls(f"{dl2Uri}/sample/仕入先")

In [0]:
#得意先
dbutils.fs.ls(f"{dl2Uri}/sample/得意先")

In [0]:
#社員
dbutils.fs.ls(f"{dl2Uri}/sample/社員")

In [0]:
#都道府県
dbutils.fs.ls(f"{dl2Uri}/sample/都道府県")

In [0]:
#商品
dbutils.fs.ls(f"{dl2Uri}/sample/商品")

In [0]:
#商品区分
dbutils.fs.ls(f"{dl2Uri}/sample/商品区分")

In [0]:
#運送会社
dbutils.fs.ls(f"{dl2Uri}/sample/運送会社")

In [0]:
#受注
dbutils.fs.ls(f"{dl2Uri}/sample/受注")

In [0]:
#受注明細
dbutils.fs.ls(f"{dl2Uri}/sample/受注明細")

###CSVファイルの読み込み

In [0]:
shiresaki = spark.read.format("csv").option("header", "false").option("mode", "DROPMALFORMED").load(f"{dl2Uri}/sample/仕入先/")
zyuchu = spark.read.format("csv").option("header", "false").option("mode", "DROPMALFORMED").load(f"{dl2Uri}/sample/受注/")
zyuchumeisai = spark.read.format("csv").option("header", "false").option("mode", "DROPMALFORMED").load(f"{dl2Uri}/sample/受注明細/")
shohin = spark.read.format("csv").option("header", "false").option("mode", "DROPMALFORMED").load(f"{dl2Uri}/sample/商品/")
shohinkubun = spark.read.format("csv").option("header", "false").option("mode", "DROPMALFORMED").load(f"{dl2Uri}/sample/商品区分/")
tokuisaki = spark.read.format("csv").option("header", "false").option("mode", "DROPMALFORMED").load(f"{dl2Uri}/sample/得意先/")
shain = spark.read.format("csv").option("header", "false").option("mode", "DROPMALFORMED").load(f"{dl2Uri}/sample/社員/")
unso = spark.read.format("csv").option("header", "false").option("mode", "DROPMALFORMED").load(f"{dl2Uri}/sample/運送会社/")
todofuken = spark.read.format("csv").option("header", "false").option("mode", "DROPMALFORMED").load(f"{dl2Uri}/sample/都道府県/")

###カラム名の設定

In [0]:
shiresaki = shiresaki.withColumnRenamed("_c0", "仕入先コード") \
                     .withColumnRenamed("_c1", "フリガナ") \
                     .withColumnRenamed("_c2", "仕入先名") \
                     .withColumnRenamed("_c3", "担当者名") \
                     .withColumnRenamed("_c4", "部署") \
                     .withColumnRenamed("_c5", "郵便番号") \
                     .withColumnRenamed("_c6", "トドウフケン") \
                     .withColumnRenamed("_c7", "都道府県") \
                     .withColumnRenamed("_c8", "住所1") \
                     .withColumnRenamed("_c9", "住所2") \
                     .withColumnRenamed("_c10", "電話番号") \
                     .withColumnRenamed("_c11", "ファクシミリ") \
                     .withColumnRenamed("_c12", "ホームページ")

zyuchu = zyuchu.withColumnRenamed("_c0", "受注コード") \
               .withColumnRenamed("_c1", "得意先コード") \
               .withColumnRenamed("_c2", "社員コード") \
               .withColumnRenamed("_c3", "出荷先名") \
               .withColumnRenamed("_c4", "出荷先郵便番号") \
               .withColumnRenamed("_c5", "出荷先都道府県") \
               .withColumnRenamed("_c6", "出荷先住所1") \
               .withColumnRenamed("_c7", "出荷先住所2") \
               .withColumnRenamed("_c8", "運送区分") \
               .withColumnRenamed("_c9", "受注日") \
               .withColumnRenamed("_c10", "締切日") \
               .withColumnRenamed("_c11", "出荷日") \
               .withColumnRenamed("_c12", "運送料")


zyuchumeisai = zyuchumeisai.withColumnRenamed("_c0", "受注コード") \
                           .withColumnRenamed("_c1", "商品コード") \
                           .withColumnRenamed("_c2", "単価") \
                           .withColumnRenamed("_c3", "数量") \
                           .withColumnRenamed("_c4", "割引")


shohin = shohin.withColumnRenamed("_c0", "商品コード") \
               .withColumnRenamed("_c1", "フリガナ") \
               .withColumnRenamed("_c2", "商品名") \
               .withColumnRenamed("_c3", "仕入先コード") \
               .withColumnRenamed("_c4", "区分コード") \
               .withColumnRenamed("_c5", "梱包単位") \
               .withColumnRenamed("_c6", "単価") \
               .withColumnRenamed("_c7", "在庫") \
               .withColumnRenamed("_c8", "発注済") \
               .withColumnRenamed("_c9", "発注点") \
               .withColumnRenamed("_c10", "生産中止")


shohinkubun = shohinkubun.withColumnRenamed("_c0", "区分コード") \
                         .withColumnRenamed("_c1", "区分名") \
                         .withColumnRenamed("_c2", "商品名") \
                         .withColumnRenamed("_c3", "説明") \
                         .withColumnRenamed("_c4", "図")


tokuisaki = tokuisaki.withColumnRenamed("_c0", "得意先コード") \
               .withColumnRenamed("_c1", "フリガナ") \
               .withColumnRenamed("_c2", "得意先名") \
               .withColumnRenamed("_c3", "担当者名") \
               .withColumnRenamed("_c4", "部署") \
               .withColumnRenamed("_c5", "郵便番号") \
               .withColumnRenamed("_c6", "トドウフケン") \
               .withColumnRenamed("_c7", "都道府県") \
               .withColumnRenamed("_c8", "住所1") \
               .withColumnRenamed("_c9", "住所2") \
               .withColumnRenamed("_c10", "電話番号") \
               .withColumnRenamed("_c11", "ファクシミリ")


shain = shain.withColumnRenamed("_c0", "社員コード") \
             .withColumnRenamed("_c1", "フリガナ") \
             .withColumnRenamed("_c2", "氏名") \
             .withColumnRenamed("_c3", "在籍支社") \
             .withColumnRenamed("_c4", "部署名") \
             .withColumnRenamed("_c5", "誕生日") \
             .withColumnRenamed("_c6", "入社日") \
             .withColumnRenamed("_c7", "自宅郵便番号") \
             .withColumnRenamed("_c8", "自宅都道府県") \
             .withColumnRenamed("_c9", "自宅住所1") \
             .withColumnRenamed("_c10", "自宅住所2") \
             .withColumnRenamed("_c11", "自宅電話番号") \
             .withColumnRenamed("_c11", "内線") \
             .withColumnRenamed("_c11", "写真") \
             .withColumnRenamed("_c12", "プロフィール1") \
             .withColumnRenamed("_c13", "プロフィール2") \
             .withColumnRenamed("_c14", "プロフィール3")


unso = unso.withColumnRenamed("_c0", "運送コード") \
           .withColumnRenamed("_c1", "運送会社") \
           .withColumnRenamed("_c2", "電話番号")


todofuken = todofuken.withColumnRenamed("_c0", "トドウフケン") \
           .withColumnRenamed("_c1", "都道府県") \
           .withColumnRenamed("_c2", "ローマ字") \
           .withColumnRenamed("_c3", "地域名ローマ字") \
           .withColumnRenamed("_c4", "地域")

## 型の設定

### 型変換

In [0]:
from pyspark.sql.types import IntegerType, TimestampType, DecimalType

shiresaki = shiresaki.withColumn("仕入先コード",shiresaki.仕入先コード.cast(IntegerType()))

zyuchu = zyuchu.withColumn("受注コード", zyuchu.受注コード.cast(IntegerType())) \
               .withColumn("得意先コード", zyuchu.得意先コード.cast(IntegerType())) \
               .withColumn("社員コード", zyuchu.社員コード.cast(IntegerType())) \
               .withColumn("受注日", zyuchu.受注日.cast(TimestampType())) \
               .withColumn("締切日", zyuchu.締切日.cast(TimestampType())) \
               .withColumn("出荷日", zyuchu.出荷日.cast(TimestampType())) \
               .withColumn("運送料", zyuchu.運送料.cast(DecimalType()))

zyuchumeisai = zyuchumeisai.withColumn("受注コード", zyuchumeisai.受注コード.cast(IntegerType())) \
                           .withColumn("商品コード", zyuchumeisai.商品コード.cast(IntegerType())) \
                           .withColumn("単価", zyuchumeisai.単価.cast(DecimalType())) \
                           .withColumn("数量", zyuchumeisai.数量.cast(IntegerType())) \
                           .withColumn("割引", zyuchumeisai.割引.cast(DecimalType()))

shohin = shohin.withColumn("商品コード", shohin.商品コード.cast(IntegerType())) \
               .withColumn("仕入先コード", shohin.仕入先コード.cast(IntegerType())) \
               .withColumn("区分コード", shohin.区分コード.cast(IntegerType())) \
               .withColumn("単価", shohin.単価.cast(DecimalType())) \
               .withColumn("在庫", shohin.在庫.cast(IntegerType()))


shohinkubun = shohinkubun.withColumn("区分コード", shohinkubun.区分コード.cast(IntegerType()))

tokuisaki = tokuisaki.withColumn("得意先コード", tokuisaki.得意先コード.cast(IntegerType()))

shain = shain.withColumn("社員コード", shain.社員コード.cast(IntegerType())) \
             .withColumn("誕生日", shain.誕生日.cast(TimestampType())) \
             .withColumn("入社日", shain.入社日.cast(TimestampType()))

unso = unso.withColumn("運送コード", unso.運送コード.cast(IntegerType()))

##データの表示

##仕入先

In [0]:
display(shiresaki)

###得意先

In [0]:
display(tokuisaki)

###社員

In [0]:
display(shain)

###都道府県

In [0]:
display(todofuken)

###商品

In [0]:
display(shohin)

###商品区分

In [0]:
display(shohinkubun)

###運送会社

In [0]:
display(unso)

###受注

In [0]:
display(zyuchu)

###受注明細

In [0]:
display(zyuchumeisai)

##出力操作

###parquet ファイルへ書き出し

In [0]:
shiresaki.write.mode('overwrite').parquet(f"{dl2Uri}/adb/parquet/仕入先/")
zyuchu.write.mode('overwrite').parquet(f"{dl2Uri}/adb/parquet/受注/")
zyuchumeisai.write.mode('overwrite').parquet(f"{dl2Uri}/adb/parquet/受注明細/")
shohin.write.mode('overwrite').parquet(f"{dl2Uri}/adb/parquet/商品/")
shohinkubun.write.mode('overwrite').parquet(f"{dl2Uri}/adb/parquet/商品区分/")
tokuisaki.write.mode('overwrite').parquet(f"{dl2Uri}/adb/parquet/得意先/")
shain.write.mode('overwrite').parquet(f"{dl2Uri}/adb/parquet/社員/")
unso.write.mode('overwrite').parquet(f"{dl2Uri}/adb/parquet/運送会社/")
todofuken.write.mode('overwrite').parquet(f"{dl2Uri}/adb/parquet/都道府県/")

##確認操作

### parquetファイルの読み出し

In [0]:
parquet_shiresaki = spark.read.format("parquet").load(f"{dl2Uri}/adb/parquet/仕入先/")
parquet_zyuchu = spark.read.format("parquet").load(f"{dl2Uri}/adb/parquet/受注/")
parquet_zyuchumeisai = spark.read.format("parquet").load(f"{dl2Uri}/adb/parquet/受注明細/")
parquet_shohin = spark.read.format("parquet").load(f"{dl2Uri}/adb/parquet/商品/")
parquet_shohinkubun = spark.read.format("parquet").load(f"{dl2Uri}/adb/parquet/商品区分/")
parquet_tokuisaki = spark.read.format("parquet").load(f"{dl2Uri}/adb/parquet/得意先/")
parquet_shain = spark.read.format("parquet").load(f"{dl2Uri}/adb/parquet/社員/")
parquet_unso = spark.read.format("parquet").load(f"{dl2Uri}/adb/parquet/運送会社/")
parquet_todofuken = spark.read.format("parquet").load(f"{dl2Uri}/adb/parquet/都道府県/")

In [0]:
display(parquet_shiresaki)

### dela lake への書き出し

In [0]:
parquet_shiresaki.write.format("delta").mode('overwrite').save(f"{dl2Uri}/adb/delta/仕入先/")
parquet_zyuchu.write.format("delta").mode('overwrite').save(f"{dl2Uri}/adb/delta/受注/")
parquet_zyuchumeisai.write.format("delta").mode('overwrite').save(f"{dl2Uri}/adb/delta/受注明細/")
parquet_shohin.write.format("delta").mode('overwrite').save(f"{dl2Uri}/adb/delta/商品/")
parquet_shohinkubun.write.format("delta").mode('overwrite').save(f"{dl2Uri}/adb/delta/商品区分/")
parquet_tokuisaki.write.format("delta").mode('overwrite').save(f"{dl2Uri}/adb/delta/得意先/")
parquet_shain.write.format("delta").mode('overwrite').save(f"{dl2Uri}/adb/delta/社員/")
parquet_unso.write.format("delta").mode('overwrite').save(f"{dl2Uri}/adb/delta/運送会社/")
parquet_todofuken.write.format("delta").mode('overwrite').save(f"{dl2Uri}/adb/delta/都道府県/")